# Preparation

In [2]:
import torch
torch.__version__

'2.3.0a0+6ddf5cf85e.nv24.04'

In [3]:
import os
import pandas as pd
import numpy as np
import random
from torch import nn

from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

from model import *
from dataset import *
from train import *

In [4]:
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

Using cuda device


In [5]:
BATCH_SIZE = 256
NUMEPOCHS = 50

In [6]:
# import pandas as pd
# df = pd.read_csv("Data/ml-1m.train.rating",sep='\t', header=None, names=['uid', 'mid', 'rating', 'timestamp'], engine='python')
# df

In [7]:
# df2 = pd.read_csv("Data/ml-1m.test.negative",sep = "\t",header=None)
# df2

# Trian and Test

### Compare the three methods 

In [10]:
model_config1 = {
    'latent_dim': 8,
    "layers_num(X)" : 0,
    # 'layers': [ 32, 16, 8],
    'model_type': 'GMF'     #　MLP, NeuMF
}

model_config2 = {
    'latent_dim': 8,
    "layers_num(X)" : 3,
    'layers': [16, 32, 16, 8],   # layers[0] is the concat of latent user vector & latent item vector
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config3 = {
    'latent_dim': 8,
    "layers_num(X)" : 3,
    'layers': [16, 32, 16, 8],  # layers[0] is the concat of latent user vector & latent item vector
    'model_type': 'NeuMF'     #　MLP, NeuMF
}

model_train(model_config2, seed = 42)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'latent_dim': 8, 'layers_num(X)': 3, 'layers': [16, 32, 16, 8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [00:14<06:55, 14.33s/it]

Epoch 1, Loss: 0.1620, Hit Ratio: 0.3366, NDCG: 0.1780


  7%|▋         | 2/30 [00:28<06:35, 14.13s/it]

Epoch 2, Loss: 0.0951, Hit Ratio: 0.3955, NDCG: 0.2102


...

Epoch 28, Loss: 0.0468, Hit Ratio: 0.3925, NDCG: 0.1984


 97%|█████████▋| 29/30 [06:43<00:14, 14.01s/it]

Epoch 29, Loss: 0.0760, Hit Ratio: 0.3896, NDCG: 0.1978


100%|██████████| 30/30 [06:57<00:00, 13.91s/it]

Epoch 30, Loss: 0.0566, Hit Ratio: 0.3839, NDCG: 0.1955
Best HR: 0.4281456953642384, Best NDCG: 0.22043416353619294 at Epoch 5


for movielens

HR, NDCG

-   `bias = False` for some linear layers
    -   GMF(Factor-8, X-0): 0.3490, 0.1849
    -   MLP(Factor-8, X-3): 0.4235, 0.2279
    -   NeuMF(Factor-8, X-3): 0.4233, 0.2258

## Reproduce the ablation study --- MLP with different layers

 K: 8
 
 layers (X): 0, 1, 2, 3, 4

In [11]:
model_config1 = {
    'latent_dim': 8,
    "layers_num(X)" : 0,
    # 'layers': [16, 32, 16, 8],   # layers[0] is the concat of latent user vector & latent item vector
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config2 = {
    'latent_dim': 8,
    "layers_num(X)" : 1,
    'layers': [16, 8],   # layers[0] is the concat of latent user vector & latent item vector
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config3 = {
    'latent_dim': 8,
    "layers_num(X)" : 2,
    'layers': [16 , 16, 8],   # layers[0] is the concat of latent user vector & latent item vector
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_config4 = {
    'latent_dim': 8,
    "layers_num(X)" : 4,
    'layers': [16, 64, 32, 16, 8],   # layers[0] is the concat of latent user vector & latent item vector
    'model_type': 'MLP'     #　MLP, NeuMF
}

model_train(model_config3)


Using cuda device
Loading preprocessed data from preprocessed_data/ml.pkl
config of model: 
 {'latent_dim': 8, 'layers_num(X)': 2, 'layers': [16, 16, 8], 'model_type': 'MLP', 'num_users': 6040, 'num_items': 3706}


  3%|▎         | 1/30 [00:13<06:34, 13.60s/it]

Epoch 1, Loss: 0.1011, Hit Ratio: 0.3200, NDCG: 0.1682


  7%|▋         | 2/30 [00:27<06:20, 13.58s/it]

Epoch 2, Loss: 0.0608, Hit Ratio: 0.3866, NDCG: 0.2083


...

Epoch 28, Loss: 0.0861, Hit Ratio: 0.3988, NDCG: 0.2119


 97%|█████████▋| 29/30 [06:15<00:12, 12.93s/it]

Epoch 29, Loss: 0.1387, Hit Ratio: 0.3927, NDCG: 0.2089


100%|██████████| 30/30 [06:28<00:00, 12.96s/it]

Epoch 30, Loss: 0.0812, Hit Ratio: 0.3929, NDCG: 0.2069
Best HR: 0.420364238410596, Best NDCG: 0.22616713607956662 at Epoch 5
